In [189]:
pip install nltk

In [190]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from warnings import filterwarnings
filterwarnings('ignore')

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [191]:
# Step 1: Load your custom stopwords
# Assuming your stopwords are in a file called 'stopwords.txt'
with open('stop_words_list.py', 'r') as file:
    custom_stopwords = file.read().splitlines()

In [192]:

# Step 1: Load your dataset Assuming your dataset has two columns:  (feedback) and  (label)
df = pd.read_csv('/Users/tanishqavenkat/Documents/OSINT INSTAGRAM/text_feedback_dataset (2).csv', encoding='ISO-8859-1')


In [193]:
# Step 2: Preprocess the data
# Check the dataset structure
print(df.head())

# Ensure there are no missing values
df.dropna(inplace=True)

# Split the dataset into features (X) and labels (y)
#X = df['feedback']  # Feedback text
#y = df['label']  # Sentiment labels (e.g., Positive, Negative, Neutral)

                                            feedback     label
0  I really enjoyed the lecture, it was very info...  Positive
1  The concepts were explained clearly, I learned...  Positive
2  The course is good, but some parts were a bit ...   Neutral
3  I'm not sure if I understood the topic complet...   Neutral
4  The teacher was a bit too fast, I had trouble ...  Negative


In [194]:
# Step 4: Clean the feedback text (removing special characters, numbers, etc.)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df['cleaned_feedback'] = df['feedback'].apply(clean_text)

# Split the dataset into features (X) and labels (y)
X = df['cleaned_feedback']  # Cleaned feedback text
y = df['label']  # Sentiment labels (e.g., Positive, Negative, Neutral)

In [195]:
# Check class distribution
print("\nClass Distribution:")
print(df['label'].value_counts())


Class Distribution:
label
Positive    508
Neutral     506
Negative    499
Name: count, dtype: int64


In [196]:
print(df.head())

                                            feedback     label  \
0  I really enjoyed the lecture, it was very info...  Positive   
1  The concepts were explained clearly, I learned...  Positive   
2  The course is good, but some parts were a bit ...   Neutral   
3  I'm not sure if I understood the topic complet...   Neutral   
4  The teacher was a bit too fast, I had trouble ...  Negative   

                                    cleaned_feedback  
0  i really enjoyed the lecture it was very infor...  
1  the concepts were explained clearly i learned ...  
2  the course is good but some parts were a bit c...  
3   im not sure if i understood the topic completely  
4  the teacher was a bit too fast i had trouble k...  


In [197]:
# Step 5: Convert text data into numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=custom_stopwords)
X_tfidf = vectorizer.fit_transform(X)

In [198]:
# Check TF-IDF features
print("\nTF-IDF Features Shape:", X_tfidf.shape)
print("First Sample's TF-IDF Features:")
print(X_tfidf[0])


TF-IDF Features Shape: (1513, 867)
First Sample's TF-IDF Features:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8 stored elements and shape (1, 867)>
  Coords	Values
  (0, 607)	0.5168231581331081
  (0, 227)	0.441662339342848
  (0, 737)	0.15220370568107916
  (0, 438)	0.43155041983982456
  (0, 414)	0.2443390664275278
  (0, 824)	0.1898226655871949
  (0, 816)	0.32087586546517
  (0, 385)	0.3601817923566046


In [199]:
# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [200]:
# Step 7: Hyperparameter tuning using GridSearchCV
penalty=['l1', 'l2', 'elasticnet']
c_values=[100,10,1.0,0.1,0.01]
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
param_grid = dict(penalty=penalty, C=c_values, solver=solver)
from sklearn.model_selection import StratifiedKFold
cv=StratifiedKFold()
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=cv, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model after tuning
best_model = grid_search.best_estimator_

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The ma

In [201]:
# Step 8: Evaluate the model
y_pred = best_model.predict(X_test)

In [202]:
# Evaluate performance
print("\nBest Hyperparameters:", grid_search.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Best Hyperparameters: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}

Classification Report:
               precision    recall  f1-score   support

    Negative       0.88      0.81      0.84       113
     Neutral       0.82      0.79      0.81        92
    Positive       0.85      0.95      0.90        98

    accuracy                           0.85       303
   macro avg       0.85      0.85      0.85       303
weighted avg       0.85      0.85      0.85       303

Accuracy: 0.8514851485148515


In [203]:
# Step 8: Use the trained model to predict sentiment for new feedback
def predict_sentiment(feedback, vectorizer, model):
    # Clean the feedback text (same preprocessing as before)
    feedback_cleaned = clean_text(feedback)
    
    # Transform the cleaned feedback into TF-IDF features
    feedback_tfidf = vectorizer.transform([feedback_cleaned])
    
    # Predict the sentiment
    sentiment = model.predict(feedback_tfidf)[0]
    print("Predicted Sentiment:",sentiment)
    
    return sentiment





In [204]:
%pip install deepface
%pip install tf-keras


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [205]:
import cv2
from deepface import DeepFace

def capture_emotion():
    """
    Captures an image from the webcam when the space key is pressed,
    detects the face, analyzes the emotion, and prints the detected emotion.
    """
    # Load face cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Start capturing video
    cap = cv2.VideoCapture(0)
    print("Press 'SPACE' to capture an image and detect emotion. Press 'q' to exit.")

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        # Convert frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Draw rectangle around detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Press SPACE to Capture', frame)

        # Capture image on 'SPACE' key press
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):  # Space key pressed
            if len(faces) == 0:
                print("No face detected. Try again.")
            else:
                print("Capturing image and analyzing emotion...")
                for (x, y, w, h) in faces:
                    face_roi = frame[y:y + h, x:x + w]  # Extract face ROI

                    try:
                        # Perform emotion analysis
                        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                        emotion = result[0]['dominant_emotion']
                        print(f"Detected Emotion: {emotion}")

                        # Display detected emotion on image
                        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                        cv2.imshow("Captured Emotion", frame)
                        cv2.waitKey(2000)  # Show detected emotion for 2 seconds

                        cap.release()
                        cv2.destroyAllWindows()
                        return emotion  # Return detected emotion
                    except Exception as e:
                        print("Error analyzing emotion:", e)

        # Exit on 'q' key press
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return None  # Return None if no emotion detected



In [206]:
# Example usage
input_feedback = input("Enter feedback text: ")
sentiment = predict_sentiment(input_feedback, vectorizer, best_model)
emotion = capture_emotion()
if sentiment == 'Positive' and emotion == 'happy':
    print("The feedback is genuine")
elif sentiment == 'Negative' and emotion == 'angry':
    print("The feedback is genuine")
elif sentiment == 'Neutral' and emotion == 'neutral':
    print("The feedback is genuine")
else:
    print("The feedback is not genuine")

Predicted Sentiment: Positive
Press 'SPACE' to capture an image and detect emotion. Press 'q' to exit.
Capturing image and analyzing emotion...
Detected Emotion: sad
The feedback is not genuine
